In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms


import sys
sys.path.append("../")
from bargains_linear import * 

In [11]:
def run_test(df,print_v=False):
    #make oustide options
    df = df[ (df['s1']!=0) & (df['s1']!=1) &  (df['phi1']>=0) & (df['phi2']>=0) & (np.abs(df['phi1'])<=100) & (np.abs(df['phi2'])<=100) ]
    df = df.reindex()
    
    df['s1*phi1'] =   (df['s1'])*df['phi1']
    df['phi1*(s1+s2)'] =df['phi1'] 
    
    transformed_cols = [ 'v' ]
    
    #'s1*phi1',
    #'phi1*(s1+s2)'
    
    for col in transformed_cols:
        df[col+'^2'] = df[col]**2
        df[col+'^(-1)'] = (df[col])**(-1)

        # Perform regressions and calculate R-squared
        predictors = sm.add_constant(df[['phi1',col,col+'^2',col+'^(-1)']])
        model = sm.OLS(df['phi2'], predictors ).fit()
        print(model.summary())
    return 0
    

passive_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_3.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_3.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_3.csv',index_col=0)
print(seq_result1)


dfs = [passive_result1,active_result1,seq_result1]

for i in range(3): 
    df = dfs[i]
    df_sample = df.sample(frac=0.5)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(run_test(df_sample))


         phi1       phi2         p1         p2        s1        s2        x1  \
0   15.895086  17.166332  24.378793  25.096154  0.366454  0.317476  0.323504   
0   15.833449  17.310044  24.587799  25.420940  0.364187  0.308289  0.615606   
0   15.730349  17.419240  24.520436  25.459369  0.374763  0.310600  0.796535   
0   15.977174  17.209393  24.205303  24.916773  0.376556  0.326612  0.116100   
0   16.236203  17.180602  24.621730  25.104398  0.379152  0.344262  0.275597   
..        ...        ...        ...        ...       ...       ...       ...   
0   15.710661  17.414615  24.837873  25.749354  0.386746  0.322296  0.989597   
0   16.137556  17.438565  24.446891  25.209631  0.369941  0.317601  0.057233   
0   15.506504  17.196072  24.626039  25.574657  0.370012  0.306070  0.241344   
0   15.741057  17.335776  24.731851  25.613822  0.374115  0.313616  0.513819   
0   16.090739  17.262358  24.171306  24.825968  0.381178  0.334399  0.018326   

          x2          v  
0   0.323504 

In [9]:
passive_result1 = pd.read_csv('../GMM_est/fake_data_Wtp/passive_data_3.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data_Wtp/active_data_3.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data_Wtp/seq_data_3.csv',index_col=0)

dfs = [passive_result1,active_result1,seq_result1]


for i in range(3): 
    df = dfs[i]
    df_sample = df.sample(frac=0.5)
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(run_test(df_sample))

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
                            OLS Regression Results                            
Dep. Variable:                   phi2   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.590
Method:                 Least Squares   F-statistic:                     178.5
Date:                Mon, 19 Feb 2024   Prob (F-statistic):           3.37e-94
Time:                        17:27:31   Log-Likelihood:                -783.63
No. Observations:                 495   AIC:                             1577.
Df Residuals:                     490   BIC:                             1598.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11

In [4]:
# Create an empty DataFrame to store the results
results_list = []

for j in [0,1,3,4]:
    # Read the dataframes
    seq_result = pd.read_csv('../GMM_est/fake_data_wtp/seq_data_%s.csv' % j, index_col=0)
    active_result = pd.read_csv('../GMM_est/fake_data_wtp/active_data_%s.csv' % j, index_col=0)
    passive_result = pd.read_csv('../GMM_est/fake_data_wtp/passive_data_%s.csv' % j, index_col=0)

    # Initialize rejection count variables
    rejections_seq = np.array([0,0,0,0])
    rejections_active = np.array([0,0,0,0])
    rejections_passive = np.array([0,0,0,0])
    
    nsims = 1000
    
    for i in range(nsims):
        # Resample from seq dataframe
        subsample_seq = seq_result.sample(frac=0.5)
        result_seq = run_test(subsample_seq)
        rejections_seq[result_seq] = rejections_seq[result_seq] + 1
        
        
        # Resample from active dataframe
        subsample_active = active_result.sample(frac=0.5)
        result_active = run_test(subsample_active)
        rejections_active[result_active] = rejections_active[result_active] + 1

        # Resample from passive dataframe
        subsample_passive = passive_result.sample(frac=0.5)
        result_passive = run_test(subsample_passive)
        rejections_passive[result_passive] = rejections_passive[result_passive]+ 1

    
    rejections_seq = rejections_seq/nsims
    rejections_active = rejections_active/nsims
    rejections_passive = rejections_passive/nsims
    
    
    # Add the results to the DataFrame
    results_list.append({'Round': j, 'DataFrame': 'passive', 'Passive Selected': rejections_passive[0],
                         'Active Selected': rejections_passive[1], 'Seq Selected': rejections_passive[2],
                         'No Selection': rejections_passive[3]})
    results_list.append({'Round': j, 'DataFrame': 'active', 'Passive Selected': rejections_active[0],
                         'Active Selected': rejections_active[1], 'Seq Selected': rejections_active[2],
                         'No Selection': rejections_active[3]})
    results_list.append({'Round': j, 'DataFrame': 'seq', 'Passive Selected': rejections_seq[0],
                         'Active Selected': rejections_seq[1], 'Seq Selected': rejections_seq[2],
                         'No Selection': rejections_seq[3]})
    
 
results_df = pd.DataFrame(data=results_list,columns=['Round','DataFrame','No Selection','Passive Selected','Active Selected','Seq Selected'])
print(results_df)

    Round DataFrame  No Selection  Passive Selected  Active Selected  \
0       0   passive         0.000               1.0              0.0   
1       0    active         0.000               0.0              1.0   
2       0       seq         0.000               0.0              0.0   
3       1   passive         0.000               1.0              0.0   
4       1    active         0.000               0.0              1.0   
5       1       seq         0.044               0.0              0.0   
6       3   passive         0.000               1.0              0.0   
7       3    active         0.000               0.0              1.0   
8       3       seq         0.000               0.0              0.0   
9       4   passive         0.000               1.0              0.0   
10      4    active         0.000               0.0              1.0   
11      4       seq         0.000               0.0              0.0   

    Seq Selected  
0          0.000  
1          0.000  
2     

In [5]:
# Create an empty DataFrame to store the results
results_list = []

for j in [0,3,4]:
    # Read the dataframes
    seq_result = pd.read_csv('../GMM_est/fake_data_wtp_logit/seq_data_%s.csv' % j, index_col=0)
    active_result = pd.read_csv('../GMM_est/fake_data_wtp_logit/active_data_%s.csv' % j, index_col=0)
    passive_result = pd.read_csv('../GMM_est/fake_data_wtp_logit/passive_data_%s.csv' % j, index_col=0)

    # Initialize rejection count variables
    rejections_seq = np.array([0,0,0,0])
    rejections_active = np.array([0,0,0,0])
    rejections_passive = np.array([0,0,0,0])

    nsims = 1000
    
    for i in range(nsims):
        # Resample from seq dataframe
        subsample_seq = seq_result.sample(frac=0.5)
        result_seq = run_test_logit(subsample_seq)
        rejections_seq[result_seq] = rejections_seq[result_seq] + 1
        
        
        # Resample from active dataframe
        subsample_active = active_result.sample(frac=0.5)
        result_active = run_test_logit(subsample_active)
        rejections_active[result_active] = rejections_active[result_active] + 1

        # Resample from passive dataframe
        subsample_passive = passive_result.sample(frac=0.5)
        result_passive = run_test_logit(subsample_passive)
        rejections_passive[result_passive] = rejections_passive[result_passive]+ 1

    
    rejections_seq = rejections_seq/nsims
    rejections_active = rejections_active/nsims
    rejections_passive = rejections_passive/nsims
    
    
    # Add the results to the DataFrame
    results_list.append({'Round': j, 'DataFrame': 'passive', 'Passive Selected': rejections_passive[0],
                         'Active Selected': rejections_passive[1], 'Seq Selected': rejections_passive[2],
                         'No Selection': rejections_passive[3]})
    results_list.append({'Round': j, 'DataFrame': 'active', 'Passive Selected': rejections_active[0],
                         'Active Selected': rejections_active[1], 'Seq Selected': rejections_active[2],
                         'No Selection': rejections_active[3]})
    results_list.append({'Round': j, 'DataFrame': 'seq', 'Passive Selected': rejections_seq[0],
                         'Active Selected': rejections_seq[1], 'Seq Selected': rejections_seq[2],
                         'No Selection': rejections_seq[3]})
    
 
results_df = pd.DataFrame(data=results_list,columns=['Round','DataFrame','No Selection','Passive Selected','Active Selected','Seq Selected'])
print(results_df)

   Round DataFrame  No Selection  Passive Selected  Active Selected  \
0      0   passive         0.019             0.981              0.0   
1      0    active         0.000             0.000              1.0   
2      0       seq         0.000             0.000              1.0   
3      3   passive         0.129             0.871              0.0   
4      3    active         0.000             0.000              1.0   
5      3       seq         0.000             0.000              1.0   
6      4   passive         0.005             0.995              0.0   
7      4    active         0.000             0.000              1.0   
8      4       seq         0.000             0.000              1.0   

   Seq Selected  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  
5           0.0  
6           0.0  
7           0.0  
8           0.0  
